Preparação dos arquivos

In [1]:
from modules.utilities import prepare_data_filesystem

prepare_data_filesystem()

Etapa Landing (Extraindo a lista de IDs)

In [2]:
from modules.etl import Landing

Landing.full_process(database='nucleotide',term='Chikungunya',retmax=1000,datetype="pdat",reldate=60)

Etapa bronze (Extraindo as informações dos IDs do XML salvo pela etapa Landing)

In [3]:
from modules.etl import Bronze

Bronze.full_process('nucleotide',rettype='gbc',retmode='xml')

Etapa Silver (Data Quality e Data Clean)